In [1]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib", "https://pypi.org/simple")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1641979226506_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl (10.5 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)

  Using cached matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl (11.5 MB)
  Using cached Pillow-8.4.0-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached kiwisolver-1.3.1-cp36-cp36m-manylinux1_x86_64.whl (1.1 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached pyparsing-3.0.6-py3-none-any.whl (97 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.5
    Not uninstalling numpy at /usr/local/lib64/python3.6/site-packages, outside environment /tmp/1642007300965-0
    Can't uninstall 'numpy'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
py

In [2]:
## loading data from s3 using keyword s3://
application= spark.read.csv('s3://sqoop.oozie.ml/application_record.csv',header=True,inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
credit= spark.read.csv('s3://sqoop.oozie.ml/credit_record.csv',header=True,inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
application.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID: integer (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)

In [5]:
application.show(20,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|ID     |CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE    |NAME_EDUCATION_TYPE          |NAME_FAMILY_STATUS  |NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|M          |Y           |Y              |0           |427500.0        |Working             |Higher education             |Civil marriage      |Rented apartment 

In [6]:
application.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

438557

In [7]:
credit.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1048575

## Questions needed to be answered

### 1. What is the proportion of females in the applicant customer base?

In [8]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
application.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID: integer (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)

In [10]:
df =application.groupby('CODE_GENDER').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.withColumn('prop_gender',f.col('count')/f.sum('count').over(Window.partitionBy())).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+------------------+
|CODE_GENDER| count|       prop_gender|
+-----------+------+------------------+
|          F|294440|0.6713836513839706|
|          M|144117|0.3286163486160294|
+-----------+------+------------------+

#### 67.14% of the applications have been filed by Females

### 2. Is homeownership higher among male applicants or female applicants?

In [12]:
application.select('NAME_HOUSING_TYPE').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|  NAME_HOUSING_TYPE|
+-------------------+
|  House / apartment|
|Municipal apartment|
|    Co-op apartment|
|   Rented apartment|
|   Office apartment|
|       With parents|
+-------------------+

##### In this House/Apartment category satisfies home ownership

In [13]:
df = application.groupby('NAME_HOUSING_TYPE').pivot('CODE_GENDER').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.withColumn('Prop_F',f.col('F')/f.sum('F').over(Window.partitionBy())).withColumn('Prop_M',f.col('M')/f.sum('M').over(Window.partitionBy())).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+------+--------------------+--------------------+
|  NAME_HOUSING_TYPE|     F|     M|              Prop_F|              Prop_M|
+-------------------+------+------+--------------------+--------------------+
|  House / apartment|267607|126224|  0.9088676810216003|  0.8758439323605126|
|Municipal apartment| 10019|  4195| 0.03402730607254449|0.029108293955605515|
|    Co-op apartment|   862|   677|0.002927591359869583|0.004697572111548...|
|   Rented apartment|  3284|  2690|0.011153375900013585|0.018665389926240485|
|   Office apartment|  2122|  1800|0.007206901236245...|0.012489851995253856|
|       With parents| 10546|  8531| 0.03581714440972694| 0.05919495965083925|
+-------------------+------+------+--------------------+--------------------+

#### 90.88% females own their home/apartment wheras 87.58% males own their homes. Hence higher among female applicants

### 3. Is there any correlation between the customer's income level and education level?

In [15]:
from pyspark.ml.stat import Correlation

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
application.select('NAME_EDUCATION_TYPE').distinct().show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+
|NAME_EDUCATION_TYPE          |
+-----------------------------+
|Academic degree              |
|Incomplete higher            |
|Secondary / secondary special|
|Lower secondary              |
|Higher education             |
+-----------------------------+

In [17]:
## Since NAME_EDUCATION_TYPE is string, need to create a numeric column in ascending order
from pyspark.sql.functions import when
application = application.withColumn('Education_level',when(f.col('NAME_EDUCATION_TYPE')=='Lower secondary',0).when(f.col('NAME_EDUCATION_TYPE')=='Secondary / secondary special',1).when(f.col('NAME_EDUCATION_TYPE')=='Incomplete higher',2).when(f.col('NAME_EDUCATION_TYPE')=='Higher education',3).otherwise(4))
                                    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
corr = application.select('AMT_INCOME_TOTAL','Education_level')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
corr_df = corr.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
corr_df.corr()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  AMT_INCOME_TOTAL  Education_level
AMT_INCOME_TOTAL          1.000000         0.221403
Education_level           0.221403         1.000000

#### There is a weak correlation of 0.22 between AMT_INCOME_TOTAL and Education

### 4. What is the average and median salary of the applicant base?

In [21]:
application.agg({'AMT_INCOME_TOTAL':'mean'}).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+
|avg(AMT_INCOME_TOTAL)|
+---------------------+
|   187524.28600950394|
+---------------------+

In [22]:
application.approxQuantile('AMT_INCOME_TOTAL',[0.5],0)[0]
## Median can be found by using approxQuantile , which returns a list. Hence, [0] to retrive value

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

160780.5

#### Mean of Income is 187524.28 and the median is 160780.5

Other questions are related to Good and Bad customers , hence need to analyze the Credit data and create target variable

In [23]:
credit.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID: integer (nullable = true)
 |-- MONTHS_BALANCE: integer (nullable = true)
 |-- STATUS: string (nullable = true)

In [24]:
credit.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
|5001712|            -1|     C|
|5001712|            -2|     C|
|5001712|            -3|     C|
|5001712|            -4|     C|
|5001712|            -5|     C|
|5001712|            -6|     C|
|5001712|            -7|     C|
|5001712|            -8|     C|
|5001712|            -9|     0|
|5001712|           -10|     0|
|5001712|           -11|     0|
|5001712|           -12|     0|
|5001712|           -13|     0|
|5001712|           -14|     0|
|5001712|           -15|     0|
+-------+--------------+------+
only showing top 20 rows

As per data dictionary , status is as

0: 1-29 days past due

1: 30-59 days past due 

2: 60-89 days overdue 

3: 90-119 days overdue 

4: 120-149 days overdue 

5: Overdue or bad debts, write-offs for more than 150 days 

C: paid off that month 

X: No loan for the month

For this, we need to consider delinquent those who have ever been overdue >=60 days. This would be 2 to 5  

In [25]:
credit_summary = credit.groupby('ID').pivot('STATUS').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
credit_summary.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+----+----+----+----+----+----+----+
|     ID|   0|   1|   2|   3|   4|   5|   C|   X|
+-------+----+----+----+----+----+----+----+----+
|5061822|   5|null|null|null|null|null|null|   1|
|5060509|  28|   1|null|null|null|null|null|   5|
|5009304|   5|null|null|null|null|null|   6|  24|
|5021819|  38|null|null|null|null|null|null|null|
|5044479|   2|null|null|null|null|null|null|   1|
|5092149|   1|null|null|null|null|null|null|   6|
|5135899|   6|null|null|null|null|null|  17|   7|
|5142868|  26|null|null|null|null|null|null|   1|
|5149573|   4|null|null|null|null|null|  33|null|
|5068077|   5|null|null|null|null|null|   1|null|
|5089546|  14|   1|null|null|null|null|null|   4|
|5090925|  10|null|null|null|null|null|null|   1|
|5112842|   6|null|null|null|null|null|   8|null|
|5005681|  13|null|null|null|null|null|  13|   5|
|5009033|   4|null|null|null|null|null|null|  13|
|5058510|null|null|null|null|null|null|   4|   7|
|5078408|   3|null|null|null|null|null|null|   7|


In [27]:
credit_summary = credit_summary.na.fill(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
credit_summary.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---+---+---+---+---+---+---+---+
|     ID|  0|  1|  2|  3|  4|  5|  C|  X|
+-------+---+---+---+---+---+---+---+---+
|5061822|  5|  0|  0|  0|  0|  0|  0|  1|
|5060509| 28|  1|  0|  0|  0|  0|  0|  5|
|5009304|  5|  0|  0|  0|  0|  0|  6| 24|
|5021819| 38|  0|  0|  0|  0|  0|  0|  0|
|5044479|  2|  0|  0|  0|  0|  0|  0|  1|
|5092149|  1|  0|  0|  0|  0|  0|  0|  6|
|5135899|  6|  0|  0|  0|  0|  0| 17|  7|
|5142868| 26|  0|  0|  0|  0|  0|  0|  1|
|5149573|  4|  0|  0|  0|  0|  0| 33|  0|
|5068077|  5|  0|  0|  0|  0|  0|  1|  0|
|5089546| 14|  1|  0|  0|  0|  0|  0|  4|
|5090925| 10|  0|  0|  0|  0|  0|  0|  1|
|5112842|  6|  0|  0|  0|  0|  0|  8|  0|
|5005681| 13|  0|  0|  0|  0|  0| 13|  5|
|5009033|  4|  0|  0|  0|  0|  0|  0| 13|
|5058510|  0|  0|  0|  0|  0|  0|  4|  7|
|5078408|  3|  0|  0|  0|  0|  0|  0|  7|
|5095205| 10|  0|  0|  0|  0|  0|  6|  0|
|5033694|  6|  0|  0|  0|  0|  0|  0|  1|
|5096014|  0|  0|  0|  0|  0|  0|  0| 18|
+-------+---+---+---+---+---+---+-

In [29]:
credit_summary = credit_summary.withColumn('Total_Delinquent',f.col('2')+f.col('3')+f.col('4')+f.col('5')).withColumn('Delinquent',when(f.col('Total_Delinquent')>0,1).otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
credit_summary.groupby('Delinquent').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|Delinquent|count|
+----------+-----+
|         1|  667|
|         0|45318|
+----------+-----+

Merging this dataframe with application df.

In [31]:
df = application.join(credit_summary.select('ID','Delinquent'),application.ID==credit_summary.ID,how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+---------------+-------+----------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|Education_level|     ID|Delinquent|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+---------------+-------+----------+
|5008804|          M|           Y|              Y|           0|        427500.0|           

In [33]:
df = df.drop('ID')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

36457

### 5. Is the proportion of bad customers higher for people who own cars?

In [35]:
car_prop = df.groupby('FLAG_OWN_CAR').agg(f.sum('Delinquent').alias('Bad_customers'),f.count('Delinquent').alias('Total_customers'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
car_prop.withColumn('prop_bad',f.col('Bad_customers')/f.col('Total_customers')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+---------------+--------------------+
|FLAG_OWN_CAR|Bad_customers|Total_customers|            prop_bad|
+------------+-------------+---------------+--------------------+
|           Y|          224|          13843|0.016181463555587663|
|           N|          392|          22614|0.017334394622800037|
+------------+-------------+---------------+--------------------+

#### 1.7% bad customers for those who don't own car and 1.6% bad customers for those who own car.
#### Hence proportion higher in those who don't
Since  the values are similar, hence not a lot of impact from this variable on the model. Maybe can be dropped later

### 6. Is the proportion of bad customers higher for those living on rent than the rest of the population?

In [37]:
renters = df.select('NAME_HOUSING_TYPE','Delinquent').withColumn('renter',when(f.col('NAME_HOUSING_TYPE')=='Rented apartment','On_rent').otherwise('Not_On_rent'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
renter_prop = renters.groupby('renter').agg(f.sum('Delinquent').alias('Bad_customers'),f.count('Delinquent').alias('Total_customers'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
renter_prop.withColumn('prop_bad',f.col('Bad_customers')/f.col('Total_customers')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+---------------+--------------------+
|     renter|Bad_customers|Total_customers|            prop_bad|
+-----------+-------------+---------------+--------------------+
|Not_On_rent|          608|          35882|0.016944428961596345|
|    On_rent|            8|            575| 0.01391304347826087|
+-----------+-------------+---------------+--------------------+

#### Proportion of bad customers on rent is 1.39% whereas for those not on rent its 1.69%. Hence is more for those not on rent

### 7. Is the proportion of bad customers higher for those who are single than married customers?

In [40]:
df.select('NAME_FAMILY_STATUS').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|  NAME_FAMILY_STATUS|
+--------------------+
|           Separated|
|             Married|
|Single / not married|
|               Widow|
|      Civil marriage|
+--------------------+

Married and Civil marriage can be considered as married customers, others as single

In [41]:
married = df.select('NAME_FAMILY_STATUS','Delinquent').withColumn('Marital_status',when(f.col('NAME_FAMILY_STATUS')=='Married','Married').when(f.col('NAME_FAMILY_STATUS')=='Civil marriage','Married').otherwise('Single'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
married_prop = married.groupby('Marital_status').agg(f.sum('Delinquent').alias('Bad_customers'),f.count('Delinquent').alias('Total_customers'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
married_prop.withColumn('prop_bad',f.col('Bad_customers')/f.col('Total_customers')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------+---------------+--------------------+
|Marital_status|Bad_customers|Total_customers|            prop_bad|
+--------------+-------------+---------------+--------------------+
|       Married|          439|          27993|0.015682492051584326|
|        Single|          177|           8464|0.020912098298676747|
+--------------+-------------+---------------+--------------------+

#### 1.56% of married customers are Delinquent but 2.09% of Single customers are delinquent. Hence Proportion higher in Single customers

In [44]:
from pyspark.sql.functions import col, countDistinct
df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+---------------+----------+
|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|Education_level|Delinquent|
+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+---------------+----------+
|          2|           2|              2|           9|             265|               5|                  5|                 5|                6|      7183|   

This indicates that the FLAG_MOBIL column has only one value. It can be dropped as will have no impact

In [45]:
df = df.drop('FLAG_MOBIL')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#### Class for creating woe and iv as given in additional section
class WOE_IV(object):
    
    """
    This class contains methods that are used for calculating the WOE and IV values in a given dataset.
    
    Attributes
    ----------
    df : spark DataFrame
        The spark dataframe
    cols_to_woe : list
        Columns that you want to convert in WOE and IV
    label_column : str
        target column of the data
    good_label : str
        Predicted variable in the form of 1 or 0
    """
    
    def __init__(self, df, cols_to_woe, label_column, good_label):
        self.df = df
        self.cols_to_woe = cols_to_woe
        self.label_column = label_column
        self.good_label = good_label
        self.fit_data = {} # This is a dictionary which will store all the WOE and IV computed in the latter part of the code

    def fit(self):  
        """
        This method takes the columns in 'cols_to_woe' and calculates the good and bad amounts, and then assigns the WOE values in the dictionary, 'fit_data'
        """
        for col_to_woe in self.cols_to_woe: # Iterating over each column in the list, cols_to_woe
            
            total_good = self.compute_total_amount_of_good() # Computes the total amount of good labels in the entire dataset
            total_bad = self.compute_total_amount_of_bad() # Computes the total amount of bad labels in the entire dataset
            
            woe_df = self.df.select(col_to_woe) # This dataframe stores the column, col_to_woe from the DataFrame
            categories = woe_df.distinct().collect() # Creates a list of all the distinct categories in the column, col_to_woe
            for category_row in categories: # Iterating over each category in the distinct categories of the column
                category = category_row[col_to_woe]
                
                # For each category, we calculate the good_amount and bad_amount
                good_amount = self.compute_good_amount(col_to_woe, category)
                bad_amount = self.compute_bad_amount(col_to_woe, category)
                
                # If the good_amount or bad_amount is 0, we replace it with 0.5, because in the calculation of woe, it comes in the denominator resulting in a Nan value which will give an error
                good_amount = good_amount if good_amount != 0 else 0.5
                bad_amount = bad_amount if bad_amount != 0 else 0.5

                # Calculating the fraction of good_amount and bad_amount specific to that category
                good_dist = good_amount / total_good
                bad_dist = bad_amount / total_bad

                # Calling the build_fit_data method which calculates the WOE and IV for the category in the column and stores it in the dictionary, fit_data
                self.build_fit_data(col_to_woe, category, good_dist, bad_dist)

    def transform(self, df):
        """
        This method creates a column called "col_to_woe + '_woe'" which replaces the given category value with the WOE value.
        It finally returns a final dataframe containing the WOE values in place of the columns
        """
        def _encode_woe(col_to_woe_):
            return F.coalesce(
                *[F.when(F.col(col_to_woe_) == category, F.lit(woe_iv['woe']))
                  for category, woe_iv in self.fit_data[col_to_woe_].items()]
            )

        for col_to_woe, woe_info in self.fit_data.items():
            # Adds another column to the df containing the final WOE values
            df = df.withColumn(col_to_woe + '_woe', _encode_woe(col_to_woe))
        return df

    def compute_total_amount_of_good(self):
     # Computes the total amount of good labels in the entire dataset
        return self.df.select(self.label_column).filter(F.col(self.label_column) == self.good_label).count()

    def compute_total_amount_of_bad(self):
     # Computes the total amount of bad labels in the entire dataset
        return self.df.select(self.label_column).filter(F.col(self.label_column) != self.good_label).count()

    def compute_good_amount(self, col_to_woe, category):
     # Computes the total amount of good labels for the specific class in the feature
        return self.df.select(col_to_woe, self.label_column)\
                      .filter(
                            (F.col(col_to_woe) == category) & (F.col(self.label_column) == self.good_label)
                      ).count()

    def compute_bad_amount(self, col_to_woe, category):
        # Computes the total amount of bad labels for the specific class in the feature
        return self.df.select(col_to_woe, self.label_column)\
                      .filter(
                            (F.col(col_to_woe) == category) & (F.col(self.label_column) != self.good_label)
                      ).count()

    def build_fit_data(self, col_to_woe, category, good_dist, bad_dist):
        """
        Calculates the WOE and value for the specific category and saves it in the woe_info dictionary which is saves in the fit_data dictionary
        
        
        Attributes:
        col_to_woe: str
        name of the feature
        
        category: str
        Class of that feature
        
        good_dist: float
        Fraction of good labels specific to that category
        
        bad_dist: float
        Fraction of bad labels specific to that category
        """
        woe_info = {
            category: {
                'woe': math.log(good_dist / bad_dist), # Calculating the WOE value for the category in the column
                'iv': (good_dist - bad_dist) * math.log(good_dist / bad_dist) # Calculating the IV value for the category in the column
            }
        }

        # Adding the feature, col_to_woe to fit_data if not present already
        if col_to_woe not in self.fit_data:
            self.fit_data[col_to_woe] = woe_info
        else:
            self.fit_data[col_to_woe].update(woe_info)

    def compute_iv(self):
        """
        Returns the IV dictionary which contains the IV value for each feature
        """
        iv_dict = {}

        for woe_col, categories in self.fit_data.items():
            iv_dict[woe_col] = 0
            for category, woe_iv in categories.items():
                iv_dict[woe_col] += woe_iv['iv'] # IV for the feature = IV value for the indiviual categories of the feature (column)
        return iv_dict

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
#Creating monotonous bins for continous variables
from pyspark.ml.feature import QuantileDiscretizer
spike_cols = ['CNT_CHILDREN','AMT_INCOME_TOTAL','DAYS_BIRTH','DAYS_EMPLOYED','CNT_FAM_MEMBERS']
max_bucket = 5
for x in spike_cols:
    r = 0
    n = max_bucket
    while ((r < 1) & (n>1)):
        discretizer = QuantileDiscretizer(numBuckets=n, inputCol=x, outputCol=x + "bucket")
        d1 = discretizer.fit(df).transform(df)
        d2 = d1.groupby(x+ "bucket").agg(f.mean(x).alias("X1"),f.mean("Delinquent").alias("Y1")).orderBy("X1")
        corr_df = d2.select('X1','Y1').toPandas()
        r=corr_df.corr(method='spearman').iloc[0,1]
        if(r == 1):
            print(x+"bucket")
            df = discretizer.fit(df).transform(df)
        elif(r<0):
            r=r*(-1)
        n = n-1
print(df.columns)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AMT_INCOME_TOTALbucket
DAYS_EMPLOYEDbucket
['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'Education_level', 'Delinquent', 'AMT_INCOME_TOTALbucket', 'DAYS_EMPLOYEDbucket']

In [48]:
df=df.na.fill('Missing') ## imputing category Missing for missing values. WOE value for this category will be calculated

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
import math
cols_to_woe = ['CODE_GENDER', 'FLAG_OWN_CAR','FLAG_OWN_REALTY','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE',
               'NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL','OCCUPATION_TYPE',
               'AMT_INCOME_TOTALbucket','DAYS_EMPLOYEDbucket']
from pyspark.sql import functions as F
woe = WOE_IV(df, cols_to_woe, 'Delinquent', 0)
woe.fit()
encoded_df = woe.transform(df)
ivs = woe.compute_iv()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
encoded_df.write.format("com.databricks.spark.csv").mode('overwrite').save('encodeddf3.csv',header = 'true') # checkpoint
encoded_df_saved = spark.read.csv("encodeddf3.csv", header=True, inferSchema=True)
print(encoded_df.count())
print(encoded_df_saved.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

36457
36457

In [52]:
from pyspark.sql import DataFrame
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import DoubleType
from pyspark.sql.types import FloatType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
features = [key+"_woe" for key,value in ivs.items() if value>0.002] ## selecting features with IV greater than 0.002
print(features)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['CODE_GENDER_woe', 'FLAG_OWN_REALTY_woe', 'NAME_INCOME_TYPE_woe', 'NAME_EDUCATION_TYPE_woe', 'NAME_FAMILY_STATUS_woe', 'NAME_HOUSING_TYPE_woe', 'OCCUPATION_TYPE_woe', 'DAYS_EMPLOYEDbucket_woe']

In [55]:
encoded_df_saved = encoded_df_saved.withColumnRenamed('Delinquent','target')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
train, test = encoded_df_saved.randomSplit([0.7, 0.3], seed = 2018)

#Creating feature vector column with all the input variables
assembler = VectorAssembler(inputCols=features, outputCol="features")
train = assembler.transform(train)
test = assembler.transform(test)

#Building logistic regression model and predict on test & train dataset using the same model 
lr = LogisticRegression(labelCol="target", featuresCol= "features",maxIter=10)
model=lr.fit(train)
predict_train=model.transform(train)
predict_test=model.transform(test)
#predict_test.select("target","prediction","probability").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
#Creating a probability column for target probability 
secondelement=udf(lambda v:float(v[1]),FloatType())
predict_train = predict_train.withColumn('prob_int', secondelement('probability'))
predict_test = predict_test.withColumn('prob_int', secondelement('probability'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
#calculating KS

#Creating deciles using the predcited probability column 
discretizer = QuantileDiscretizer(numBuckets=10, inputCol="prob_int", outputCol="PROB_BUCKET")
predict_train = discretizer.fit(predict_train).transform(predict_train)
predict_test = discretizer.fit(predict_test).transform(predict_test)

#Creating #Good & # bad for each decile 
ks_train = predict_train.groupby('PROB_BUCKET').agg(F.count("CODE_GENDER").alias("Total"), 
                                                    F.max("prob_int").alias("max_prob"), 
                                                    F.min("prob_int").alias("min_prob"),  
                                                    F.sum("target").alias("bads") ,
                                                    F.count(F.when(col("target") == 0,1)).alias("goods")).sort(col('PROB_BUCKET').desc())

ks_test = predict_test.groupby('PROB_BUCKET').agg(F.count("CODE_GENDER").alias("Total"), 
                                                  F.max("prob_int").alias("max_prob"), 
                                                  F.min("prob_int").alias("min_prob"),  
                                                  F.sum("target").alias("bads") ,
                                                  F.count(F.when(col("target") == 0,1)).alias("goods")).sort(col('PROB_BUCKET').desc())

#Creating %cumulative good,%cumulative bad & KS 
ks_train = ks_train.withColumn('%cumuBad', F.sum('bads').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0))/ F.sum('bads').over(Window.partitionBy()))
ks_train = ks_train.withColumn('%cumuGood', F.sum('goods').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0))/ F.sum('goods').over(Window.partitionBy()))
ks_train = ks_train.withColumn('%diff', F.col('%cumuBad') -  F.col('%cumuGood'))
ks_train = ks_train.withColumn('max_diff', F.when(F.col('%diff') ==F.max('%diff').over(Window.partitionBy()) ,'<----').otherwise(' '))

ks_test = ks_test.withColumn('%cumuBad', F.sum('bads').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0))/ F.sum('bads').over(Window.partitionBy()))
ks_test = ks_test.withColumn('%cumuGood', F.sum('goods').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0))/ F.sum('goods').over(Window.partitionBy()))
ks_test = ks_test.withColumn('%diff', F.col('%cumuBad') -  F.col('%cumuGood'))
ks_test = ks_test.withColumn('max_diff', F.when(F.col('%diff') ==F.max('%diff').over(Window.partitionBy()) ,'<----').otherwise(' '))

ks_train.sort(col('PROB_BUCKET').desc()).show()
ks_test.sort(col('PROB_BUCKET').desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+-----------+------------+----+-----+------------------+-------------------+--------------------+--------+
|PROB_BUCKET|Total|   max_prob|    min_prob|bads|goods|          %cumuBad|          %cumuGood|               %diff|max_diff|
+-----------+-----+-----------+------------+----+-----+------------------+-------------------+--------------------+--------+
|        9.0| 2574|0.104513474| 0.025098018|  90| 2484|0.2102803738317757|0.09911815170982802| 0.11116222212194768|        |
|        8.0| 2576|0.025017241| 0.021174047|  54| 2522|0.3364485981308411|0.19975260364710107| 0.13669599448374004|        |
|        7.0| 2504|0.021159854| 0.018497499|  46| 2458|0.4439252336448598|0.29783328678025617| 0.14609194686460364|   <----|
|        6.0| 3212| 0.01849441| 0.017233854|  53| 3159|0.5677570093457944| 0.4238857188460157|  0.1438712904997787|        |
|        5.0| 1967|0.017231567| 0.015982034|  29| 1938|0.6355140186915887| 0.5012170304457124| 0.13429698824587633|        |


In [68]:
#Creating prediction classs based on the prob cut off by KS value
from pyspark.sql.types import DoubleType
min_prob_max_ks = ks_test.where(col("max_diff") == '<----').select(col("min_prob")).collect()[0]
predict_test = predict_test.withColumn('pred_class', F.when(F.col('prob_int') > round(min_prob_max_ks[0],4) ,1).otherwise(0))
predict_test.groupBy('pred_class','target').count().show()

#Generating evaluation metrics
results = predict_test.select(['pred_class', 'target'])
results = results.withColumn("target", results["target"].cast(DoubleType()))
results = results.withColumn("pred_class", results["pred_class"].cast(DoubleType()))

predictionAndLabels=results.rdd
metrics = MulticlassMetrics(predictionAndLabels)
metrics1 = BinaryClassificationMetrics(predictionAndLabels)

cm=metrics.confusionMatrix().toArray()
print(cm)
# Overall statistics
precision = metrics.precision(1)
recall = metrics.recall(1)
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)
# Area under precision-recall curve
print("Area under PR = %s" % metrics1.areaUnderPR)
# Area under ROC curve
print("Area under ROC = %s" % metrics1.areaUnderROC)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-----+
|pred_class|target|count|
+----------+------+-----+
|         1|     0| 4661|
|         1|     1|  113|
|         0|     0| 6119|
|         0|     1|   75|
+----------+------+-----+

[[6119. 4661.]
 [  75.  113.]]
Summary Stats
Precision = 0.023669878508588187
Recall = 0.601063829787234
F1 Score = 0.5681983953318746
Area under PR = 0.0223675303669041
Area under ROC = 0.5843445308490902